<a href="https://colab.research.google.com/github/joaopcnogueira/colab-notebooks/blob/main/PSI_Population_Stability_Index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Carregando os dados

In [2]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Listando as variáveis de modelagem

In [3]:
cat_vars = ['Pclass', 'Sex', 'Embarked']
num_vars = ['Age', 'SibSp', 'Fare']
features = cat_vars + num_vars
target = 'Survived'

X = df[features].copy()
y = df[target].copy()

# Feature Engineering

In [4]:
num_pipe = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value=-999))
])

cat_pipe = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
]) 

transformers = ColumnTransformer(transformers=[
                ('numeric', num_pipe, num_vars),
                ('categoric', cat_pipe, cat_vars)
])

model = Pipeline(steps=[
        ('transformers', transformers),
        ('model', RandomForestClassifier(random_state=42, max_depth=3))
])

# Train Test Split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model Training

In [6]:
model.fit(X_train, y_train)

y_proba_train = model.predict_proba(X_train)[:,1]
y_proba_test  = model.predict_proba(X_test)[:,1]

# Model Evaluation

In [7]:
from sklearn.metrics import roc_auc_score

auc_train = roc_auc_score(y_train, y_proba_train)
auc_test  = roc_auc_score(y_test, y_proba_test)

print(f'ROCAUC Train: {auc_train:.6f}')
print(f'ROCAUC Test: {auc_test:.6f}')

ROCAUC Train: 0.857990
ROCAUC Test: 0.872072


# Population Stability Index

Função para calcular o PSI para uma única variável

In [8]:
def calculate_psi(expected, actual, buckets=10):
    '''Calculate the PSI for a single numeric variable

    Params:
    ------
        expected: pandas series or numpy array
        actual: pandas series or numpy array of new values, same size as expected
        buckets: number of percentile ranges to bucket the values into

    Returns:
    -------
       psi_value: calculated PSI value
    '''

    df_expected = pd.DataFrame({'expected': expected})
    df_actual   = pd.DataFrame({'actual': actual}) 

    df_expected['expected_bins'], bins = pd.qcut(df_expected['expected'], q=buckets, retbins=True, labels=False, duplicates='drop')
    df_actual['actual_bins'] = pd.cut(df_actual['actual'], bins=bins, labels=False)

    psi_df = (
        df_expected['expected_bins'].value_counts(1).sort_index().reset_index().rename(columns={'index': 'bins', 'expected_bins': 'prop_expected'})
        .merge(df_actual['actual_bins'].value_counts(1).sort_index().reset_index().rename(columns={'index': 'bins', 'actual_bins': 'prop_actual'}),
               on='bins', how='left')
        .assign(psi = lambda df: np.where((df['prop_actual'] == 0) | (df['prop_expected'] == 0), 0, 
                                          (df['prop_actual'] - df['prop_expected'])*np.log(df['prop_actual']/df['prop_expected'])))
    )
    
    psi_value = psi_df['psi'].sum()
    return psi_value

    return df_actual

In [10]:
calculate_psi(y_proba_train, y_proba_test)

0.06001324825109782

- PSI < 0.1 - No change. You can continue using existing model.
- PSI >= 0.1 but less than 0.2 - Slight change is required.
- PSI >= 0.2 - Significant change is required. Ideally, you should not use this model any more.

Reference: https://www.listendata.com/2015/05/population-stability-index.html

In [11]:
# funcion docstring
help(calculate_psi)

Help on function calculate_psi in module __main__:

calculate_psi(expected, actual, buckets=10)
    Calculate the PSI for a single numeric variable
    
    Params:
    ------
        expected: pandas series or numpy array
        actual: pandas series or numpy array of new values, same size as expected
        buckets: number of percentile ranges to bucket the values into
    
    Returns:
    -------
       psi_value: calculated PSI value

